### Generating 16x16 images of Sprites 🧝‍

In this notebook, you will find:
- **Principle of DDPM**
  - An explanation of the principles behind the algorithms for both the training and sampling (inference) stages, as proposed by [[1](#acknowledgements--references)].
- **Implementation of a Diffusion Model** with the following structure:
  - A UNet as the noise predictor

**Outline**:
1. [Overview](#1-overview)
2. [Why we can directly obtain a noised image $I_t$?](#2-why-we-can-directly-obtain-a-noised-image-)
3. [How to train the noise predictor](#3-how-to-train-the-noise-predictor)
4. [Implementation 1: training a UNet](#4-implementation-1-training-a-unet)
5. [Sampling](#5-sampling)
6. [Derivation of equation $(3)$](#6-derivation-of-equation)
7. [Implementation 2: sampling](#7-implementation-2-sampling)
8. [References](#acknowledgements--references)

#### 1. Overview  <a id="overview"></a>
The Denoising Diffusion Probabilistic Model (DDPM) consists of two main processes:

1. **Forward Process (Diffusion Process)**  
   In this stage, as is shown in the image, you might think that noise is gradually added to the input image.   
   However, Instead of iteratively adding noise step by step, we can directly obtain a noised image $ I_t $ from the original image $ I_0 $ using a **predefined noise schedule**.   
   *We will explain this in more detail later.*

   <img src="images\fwd.png" width="80%">

2. **Reverse Process (Denoising Process)**  
   This stage aims to remove noise from a given noisy image to recover a clean image.   
   To achieve this, we train a **Noise Predictor** (typically a neural network) that learns to predict the noise added at each step.   
   Once the noise is estimated, we subtract it from the noisy image to progressively reconstruct the original clean image.

   <img src="images\rvs.png" width="80%">

In summary:
- The forward process produces a set of progressively noised images from $ I_0 $ using a predefined sequence of noise scaling factors $ \{\alpha_t$$_{t=1}^{T} $ and Gaussian noise. Hence, this stage is fixed and does not require training.
- The reverse process involves training a Noise Predictor to learn the noise patterns effectively therefore estimating and removing the added noise.



#### 2. Why we can directly obtain a noised image $I_t$? <a id="2"></a>
Suppose we add noise step by step:  

Given a clean image $ I_0 $ and a full Gaussian noise image $ \epsilon_0 \sim \mathcal{N}(0, I) $, the first noised image can be expressed as:  

$ I_1 = \sqrt{1-\beta_1} I_0 + \sqrt{  \beta_1} \epsilon_0 $

The rest of the images will perform identically:

$ I_2 = \sqrt{1- \beta_2} I_1 + \sqrt{   \beta_2} \epsilon_1 $  
...  
$ I_t = \sqrt{1- \beta_t} I_{t-1} + \sqrt{   \beta_t} \epsilon_{t-1}$

Note that $ \beta_i $ values are **predifined hyperparameters** controlling the amount of noise added to the image at each step. As shown in the figure, if $ I_T $ is the fully noised image, $ \beta_i (i\in[1,T]) $ should **increase** progressively.


<img src="images\f1.png" width="80%">

To simplify this step-by-step noise accumulation, we analyze how the noise terms propagate. Since $ \epsilon_i, i \in [0,t-1] \sim \mathcal{N}(0, I) $, we can express the accumulated noise as:

$$ \sqrt{1-\beta_2}\sqrt{\beta_1}\epsilon_0 + \sqrt{\beta_2}\epsilon_1 = \sqrt{1-(1-\beta_1)(1-\beta_2)} \epsilon $$

Since both noise terms come from independent Gaussian distributions, their sum remains Gaussian with a new variance.
The combined variance follows from the property that multiplying independent Gaussian noise terms by deterministic coefficients results in a weighted sum of variances.


Thus, the general equation simplifies to:

$$ I_t = \sqrt{(1-\beta_1)(1-\beta_2)...(1-\beta_t)} I_0 + \sqrt{1-(1-\beta_1)(1-\beta_2)...(1-\beta_t)} \epsilon  \tag{1}$$
where $ \epsilon \sim \mathcal{N}(0, I) $.

To simplify this equation, suppose $\alpha_i = 1 - \beta_i, i\in[0,T];\bar{\alpha_t} = \alpha_1\alpha_2 ... \alpha_t$.
Hence, equation $ (1) $ can be rewritten as:
$$
I_t = \sqrt{\bar{\alpha_t}} I_0 + \sqrt{1-\bar{\alpha_t}} \epsilon \tag{2}$$
where $ \epsilon \sim \mathcal{N}(0, I)$


#### 3. How to train the Noise Predictor?
In the last section, we've learned how to obtain the noised images, which serve as the supervision (ground truth), given the clean image $ I_0 $. We need to train a **Noise Predictor** model, denoted as $ \epsilon_\theta $, to approximate the ground truth noise $ \epsilon $.

Check the algorithm proposed by [1]:

<img src="images\eq.png" width="50%">

#### 4. Implementation 1: Training a UNet
With the given algorithm, we could train an actual Noise Predictor -- UNet!  
The structure of a **simplified** UNet with input dimensions of (16,16,3) is depicted as follows:

<img src="images\sunet.png" width="60%">

The goal of a UNet is to predict noise by first downsampling the image to embeddings in the latent space, then upsampling it to predict the noise.


We can construct our neural network following this architecture.

In [4]:
from typing import Dict, Tuple
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
from IPython.display import HTML
import os
from utils import *

In [5]:
class ResidualConvBlock(nn.Module):
    def __init__(
            self, in_channels: int, out_channels: int, is_res: bool=False
    )->None:
        super().__init__()

        # Check if input and output channels are the same for the residual conection
        self.same_channels = in_channels == out_channels

        self.is_res = is_res

        # First convolutional layer
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),   
            nn.BatchNorm2d(out_channels),   
            nn.GELU(),   # TODO why GeLU
        )
        # Second convolutional layer
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.GELU()
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x1 = self.conv1(x)
        x2 = self.conv2(x1)

        if self.is_res:
            
            if self.same_channels:
                out = x + x2
            else:
                # if the dimension of input and output channels are not the same, 
                # apply a 1x1 conv layer to match dimension before res connecting
                shortcut = nn.Conv2d(x.shape[1], x2.shape[1], kernel_size = 1, stride=1, padding=0).to(x.device)
                out = shortcut(x) + x2
            # print(f"res conv forward: x {x.shape}, x1 {x1.shape}, x2 {x2.shape}, out {out.shape}")

            # normalization: divide by sqrt(2)
            return out / 1.414
        
        else:
            return x2
    
    def get_out_channels(self):
        return self.conv2[0].out_channels
    
    def set_out_channels(self, out_channels):
        self.conv1[0].out_channels = out_channels
        self.conv2[0].in_channels = out_channels
        self.conv1[0].out_channels = out_channels

In [12]:
class UnetUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__() # TODO super(UnetUp, self).__init__()
        
        # Create a list of layers for the upsampling block
        # The block consists of a ConvTranspose2d layer for upsampling, followed by two ResidualConvBlock layers
        
        self.model = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, 2, 2),
            ResidualConvBlock(out_channels, out_channels),
            ResidualConvBlock(out_channels, out_channels),
        )

    def forward(self, x, skip):
        # Concatenate the input tensor x with the skip connection tensor along the channel dimension
        # Here, we concatenate the feature map x, which has undergone downsampling operations, 
        # with the corresponding feature map skip from the encoder along the channel dimension (dim=1). 
        # By doing so, we can directly transfer the feature information extracted at different levels in the encoder to the decoder. 
        # This helps the decoder leverage more comprehensive features from the encoder when restoring image details, 
        # reducing information loss and improving the accuracy of segmentation.
        x = torch.cat((x, skip), 1)
        
        # Pass the concatenated tensor through the sequential model and return the output
        x = self.model(x)
        return x

In [6]:
class UnetDown(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UnetDown, self).__init__()
        
        # Create a list of layers for the downsampling block
        # Each block consists of two ResidualConvBlock layers, followed by a MaxPool2d layer for downsampling
        layers = [ResidualConvBlock(in_channels, out_channels), ResidualConvBlock(out_channels, out_channels), nn.MaxPool2d(2)]
        
        # Use the layers to create a sequential model
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        # Pass the input through the sequential model and return the output
        return self.model(x)

To enhance the performance and flexibility of the diffusion model, UNet can take in more informations in the form of embeddings, such as:
- Time embeddings: related to the time steps and noise level, as discussed earlier, the noise level controlling parameter $\beta$ increases with the timesteps.
- Context embeddings: provide additional context for the generation process, such as text descriptions or other conditional inputs.

In [7]:
class EmbedFC(nn.Module):
    def __init__(self, input_dim, emb_dim):
        
        super().__init__() # TODO super(EmbedFC, self).__init__()
        '''
        This class defines a generic one layer feed-forward neural network for embedding input data of
        dimensionality input_dim to an embedding space of dimensionality emb_dim.
        '''
        self.input_dim = input_dim
        
        # define the layers for the network
        layers = [
            nn.Linear(input_dim, emb_dim),
            nn.GELU(),
            nn.Linear(emb_dim, emb_dim),
        ]
        
        # create a PyTorch sequential model consisting of the defined layers
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        # flatten the input tensor
        x = x.view(-1, self.input_dim)
        # apply the model layers to the flattened tensor
        return self.model(x)

In [11]:
class ContextUnet(nn.Module):
    def __init__(self, in_channels, n_feat=256, n_cfeat=10, height=28):  # cfeat - context features
        super(ContextUnet, self).__init__()

        # number of input channels, number of intermediate feature maps and number of classes
        self.in_channels = in_channels
        self.n_feat = n_feat
        self.n_cfeat = n_cfeat
        self.h = height  #assume h == w. must be divisible by 4, so 28,24,20,16...

        # Initialize the initial convolutional layer
        self.init_conv = ResidualConvBlock(in_channels, n_feat, is_res=True)

        # Initialize the down-sampling path of the U-Net with two levels
        self.down1 = UnetDown(n_feat, n_feat)        # down1 #[10, 256, 8, 8]
        self.down2 = UnetDown(n_feat, 2 * n_feat)    # down2 #[10, 256, 4,  4]
        
         # original: self.to_vec = nn.Sequential(nn.AvgPool2d(7), nn.GELU())
        self.to_vec = nn.Sequential(
            nn.AvgPool2d((4)), 
            nn.GELU()
        )

        # Embed the timestep and context labels with a one-layer fully connected neural network
        self.timeembed1 = EmbedFC(1, 2*n_feat)
        self.timeembed2 = EmbedFC(1, 1*n_feat)
        self.contextembed1 = EmbedFC(n_cfeat, 2*n_feat)
        self.contextembed2 = EmbedFC(n_cfeat, 1*n_feat)

        # Initialize the up-sampling path of the U-Net with three levels
        self.up0 = nn.Sequential(
            nn.ConvTranspose2d(2 * n_feat, 2 * n_feat, self.h//4, self.h//4), # up-sample  
            nn.GroupNorm(8, 2 * n_feat), # normalize                       
            nn.ReLU(),
        )
        self.up1 = UnetUp(4 * n_feat, n_feat)
        self.up2 = UnetUp(2 * n_feat, n_feat)

        # Initialize the final convolutional layers to map to the same number of channels as the input image
        self.out = nn.Sequential(
            nn.Conv2d(2 * n_feat, n_feat, 3, 1, 1), # reduce number of feature maps   #in_channels, out_channels, kernel_size, stride=1, padding=0
            nn.GroupNorm(8, n_feat), # normalize
            nn.ReLU(),
            nn.Conv2d(n_feat, self.in_channels, 3, 1, 1), # map to same number of channels as input
        )

    def forward(self, x, t, c=None):
        """
        x : (batch, n_feat, h, w) : input image
        t : (batch, n_cfeat)      : time step
        c : (batch, n_classes)    : context label
        """
        # x is the input image, c is the context label, t is the timestep, context_mask says which samples to block the context on

        # pass the input image through the initial convolutional layer
        x = self.init_conv(x)
        # pass the result through the down-sampling path
        down1 = self.down1(x)       #[10, 256, 8, 8]
        down2 = self.down2(down1)   #[10, 256, 4, 4]
        
        # convert the feature maps to a vector and apply an activation
        hiddenvec = self.to_vec(down2)
        
        # mask out context if context_mask == 1
        if c is None:
            c = torch.zeros(x.shape[0], self.n_cfeat).to(x)
            
        # embed context and timestep
        cemb1 = self.contextembed1(c).view(-1, self.n_feat * 2, 1, 1)     # (batch, 2*n_feat, 1,1)
        temb1 = self.timeembed1(t).view(-1, self.n_feat * 2, 1, 1)
        cemb2 = self.contextembed2(c).view(-1, self.n_feat, 1, 1)
        temb2 = self.timeembed2(t).view(-1, self.n_feat, 1, 1)
        #print(f"uunet forward: cemb1 {cemb1.shape}. temb1 {temb1.shape}, cemb2 {cemb2.shape}. temb2 {temb2.shape}")


        up1 = self.up0(hiddenvec)
        up2 = self.up1(cemb1*up1 + temb1, down2)  # add and multiply embeddings
        up3 = self.up2(cemb2*up2 + temb2, down1)
        out = self.out(torch.cat((up3, x), 1))
        return out


In [8]:
# hyperparameters

# diffusion hyperparameters
timesteps = 500
beta1 = 1e-4
beta2 = 0.02

# network hyperparameters
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device('cpu'))
n_feat = 64 # 64 hidden dimension feature
n_cfeat = 5 # context vector is of size 5
height = 16 # 16x16 image
save_dir = './weights/'

# training hyperparameters
batch_size = 100
n_epoch = 32
lrate=1e-3

In [9]:
# construct DDPM noise schedule
b_t = (beta2 - beta1) * torch.linspace(0, 1, timesteps + 1, device=device) + beta1
a_t = 1 - b_t
ab_t = torch.cumsum(a_t.log(), dim=0).exp()    
ab_t[0] = 1

In [13]:
# construct model
nn_model = ContextUnet(in_channels=3, n_feat=n_feat, n_cfeat=n_cfeat, height=height).to(device)

1. load the pretrained model without training ...

In [14]:
# load in model weights and set to eval mode
nn_model.load_state_dict(torch.load(f"{save_dir}/context_model_trained.pth", map_location=device))
nn_model.eval()
print("Loaded in Model")

Loaded in Model


2. Train it anyway ...

In [33]:
transform = transforms.Compose([
    transforms.ToTensor(), 
    # from [0,255] to range [0.0,1.0]
    transforms.Normalize((0.5,), (0.5,))  
    # range [-1,1]

])

# load dataset and construct optimizer
dataset = CustomDataset("./data/sprites_1788_16x16.npy", "./data/sprite_labels_nc_1788_16x16.npy", transform, null_context=False)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=1)
optim = torch.optim.Adam(nn_model.parameters(), lr=lrate)

sprite shape: (89400, 16, 16, 3)
labels shape: (89400, 5)


In [34]:
# helper function: perturbs an image to a specified noise level
def perturb_input(x, t, noise):
    return ab_t.sqrt()[t, None, None, None] * x + (1 - ab_t[t, None, None, None]) * noise

In [35]:
# training without context code
save_dir = './weights/train_0316'
# set into train mode
nn_model.train()

for ep in range(n_epoch):
    print(f'epoch {ep}')
    
    # linearly decay learning rate
    optim.param_groups[0]['lr'] = lrate*(1-ep/n_epoch)
    
    pbar = tqdm(dataloader, mininterval=2 )
    for x, _ in pbar:   # x: images
        optim.zero_grad()
        x = x.to(device)
        
        # perturb data
        noise = torch.randn_like(x)
        t = torch.randint(1, timesteps + 1, (x.shape[0],)).to(device) # timestap
        x_pert = perturb_input(x, t, noise)
        
        # use network to recover noise
        pred_noise = nn_model(x_pert, t / timesteps) 
        
        # loss is mean squared error between the predicted and true noise
        loss = F.mse_loss(pred_noise, noise)
        loss.backward()
        
        optim.step()

    # save model periodically
    if ep%4==0 or ep == int(n_epoch-1):
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        torch.save(nn_model.state_dict(), save_dir + f"model_{ep}.pth")
        print('saved model at ' + save_dir + f"model_{ep}.pth")

epoch 0


100%|██████████| 894/894 [00:38<00:00, 22.93it/s]


saved model at ./weights/train_0316model_0.pth
epoch 1


100%|██████████| 894/894 [00:42<00:00, 21.16it/s]


epoch 2


...

epoch 30


100%|██████████| 894/894 [00:39<00:00, 22.58it/s]


epoch 31


100%|██████████| 894/894 [00:41<00:00, 21.55it/s]

saved model at ./weights/train_0316model_31.pth


#### 5. SAMPLING
After training a noise predictor, we can perform inference (i.e., generate an image) using a noisy image $x_T$ and our noise predictor by denoising it step by step.  
The goal of this stage is to approximate the probability distribution $ P_\theta(x) $ and make it as close as possible to the real image distribution $ P_{data}(x) $.  
The framework is illustrated below:

<img src="images/sampling.png" width="80%">

The discrepancy between $ P_\theta(x) $ and $ P_{data}(x) $ can be measured using the KL divergence $ KL(P_{data} \| P_\theta) $, which is minimized during training.

During sampling, we generate an image by reversing the forward diffusion process:

$$
P_\theta(x_0) = \int_{x_1}^{x_T} P(x_T) P_\theta(x_{T-1}|x_T) ... P_\theta(x_0 | x_1) dx_1 ... dx_T \tag{3}
$$

where:
- $ P(x_T) $ is a prior Gaussian distribution.
- $ P_\theta(x_{t-1} | x_t) $ represents the learned reverse process, which removes noise at each step.

To generate an image:
1. **Start from Gaussian noise**: Sample $ x_T \sim \mathcal{N}(0, I) $.
2. **Apply denoising steps iteratively**: Use the noise predictor $ \epsilon_\theta(x_t, t) $ to estimate the noise and remove it step by step until we obtain $ x_0 $.
3. **Final output**: The reconstructed clean image $ x_0 $ represents the generated image.

The denoising process can be formulated as:

$$
x_{t-1} = \frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \epsilon_\theta(x_t, t) \right) + \sigma_t z
$$

where:
- $ \alpha_t $ and $ \bar{\alpha}_t $ are the noise scheduling coefficients.
- $ \epsilon_\theta(x_t, t) $ is the predicted noise.
- $ \sigma_t z $ is an additional stochastic term (optional for improved diversity).

By iteratively applying this equation, we obtain a high-quality image from pure Gaussian noise.


#### 6. Derivation of Equation $(3)$

Suppose the forward diffusion process is given by:

$$
q(x_t | x_0) = \sqrt{\bar{\alpha}_t} \cdot x_0 + \sqrt{1 - \bar{\alpha}_t} \cdot \epsilon,
$$

where $\epsilon$ represents the predicted noise.

The maximum likelihood estimation of $ P_\theta(x_0) $ is given by:

$$
\log P(x) \geq \mathbb{E}_{q(x_1:x_T | x_0)} \left[ \log \left(\frac{P(x_0:x_T)}{q(x_1:x_T | x_0)} \right) \right],
$$

where

$$
q(x_1:x_T | x_0) = q(x_1 | x_0) q(x_2 | x_1) \dots q(x_T | x_{T-1}).
$$

The derivation is illustrated in the figure below [2]:

<img src=".\images\d2.png" width="80%">


<img src=".\images\d3.png" width="80%">

We obtain:

$$
\mathbb{E}_{q(x_1 | x_0)} [\log P(x_0 | x_1)] 
- KL(q(x_T | x_0) || P(x_T))
- \sum_{t=2}^{T} \mathbb{E}_{q(x_t | x_0)} \left[ KL(q(x_{t-1} | x_t, x_0) || P(x_{t-1} | x_t)) \right]. \tag{4}
$$

Only the *denoise matching term* is related to our trainable parameters, hence we need to minize the term:

$$
\sum_{t=2}^{T} \mathbb{E}_{q(x_t | x_0)} \left[ KL(q(x_{t-1} | x_t, x_0) || P(x_{t-1} | x_t)) \right] \tag{5}
$$

Since:

$$
q(x_{t-1} | x_t, x_0) = \frac{q(x_{t-1}, x_t | x_0)}{q(x_t | x_0)}
= \frac{q(x_t | x_{t-1}) q(x_{t-1} | x_0)}{q(x_t | x_0)},
$$

and since both the terms $q(x_t | x_{t-1})$, $q(x_{t-1} | x_0)$ and $q(x_t | x_0)$ are Gaussian distributions, $ q(x_{t-1} | x_t, x_0) $ is also a Gaussian distribution.

Through further derivation [2]:

<img src=".\images\d4.png" width="80%">

We find that $ q(x_{t-1} | x_t, x_0) $ follows a Gaussian distribution with:

- **Mean** = $$\frac{\sqrt{\bar{\alpha_{t-1}}}\beta_t x_0 + \sqrt{\alpha_t}(1-\bar{\alpha_{t-1}})x_t}{1-\bar{\alpha_t}} \tag{6}$$
- **Variance** = $$\frac{1-\bar{\alpha_{t-1}}}{1-\bar{\alpha_t}}$$


##### **KL Divergence Minimization**  
Now, our goal is to minimize the KL divergence between $ q(x_{t-1} | x_t, x_0) $ and $ P(x_{t-1} | x_t) $ in Equation $(5)$.  

Both of these terms follow **Gaussian distributions**. The mean and variance of $ q(x_{t-1} | x_t, x_0) $ are **fixed** (as shown earlier), and the variance of $ P(x_{t-1} | x_t) $ is also fixed, but its mean is **trainable**.    
Thus, our objective is to make the mean of $ P(x_{t-1} | x_t) $ as close as possible to that of $ q(x_{t-1} | x_t, x_0) $.

Recalling Equation $(6)$, the mean of $ q(x_{t-1} | x_t, x_0) $ is:  
$$
\frac{\sqrt{\bar{\alpha}_{t-1}} \beta_t x_0 + \sqrt{\alpha_t} (1 - \bar{\alpha}_{t-1}) x_t}{1 - \bar{\alpha}_t}.
$$  
From the noise prediction process, we have:
$$
x_t = \sqrt{\bar{\alpha}_t} \cdot x_0 + \sqrt{1 - \bar{\alpha}_t} \cdot \epsilon,
$$
which leads to:
$$
x_0 = \frac{x_t - \sqrt{1 - \bar{\alpha}_t} \cdot \epsilon}{\sqrt{\bar{\alpha}_t}}.
$$
Substituting this into Equation $(6)$, we obtain:
$$
\frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \epsilon_\theta(x_t, t) \right).
$$

#### 7. Implementation 2: Sampling

In [15]:
# helper function; removes the predicted noise (but adds some noise back in to avoid collapse)
def denoise_add_noise(x, t, pred_noise, z=None):
    if z is None:
        z = torch.randn_like(x)
    noise = b_t.sqrt()[t] * z
    mean = (x - pred_noise * ((1 - a_t[t]) / (1 - ab_t[t]).sqrt())) / a_t[t].sqrt()
    return mean + noise

In [16]:
# load in model weights and set to eval mode
nn_model.load_state_dict(torch.load(f"./weights/train_0316model_31.pth", map_location=device))
nn_model.eval()
print("Loaded in Model")

Loaded in Model


In [17]:
# sample using standard algorithm
@torch.no_grad()
def sample_ddpm(n_sample, save_rate=20):
    # x_T ~ N(0, 1), sample initial noise
    samples = torch.randn(n_sample, 3, height, height).to(device)  

    # array to keep track of generated steps for plotting
    intermediate = [] 
    for i in range(timesteps, 0, -1):
        print(f'sampling timestep {i:3d}', end='\r')

        # reshape time tensor
        t = torch.tensor([i / timesteps])[:, None, None, None].to(device)

        # sample some random noise to inject back in. For i = 1, don't add back in noise
        z = torch.randn_like(samples) if i > 1 else 0

        eps = nn_model(samples, t)    # predict noise e_(x_t,t)
        samples = denoise_add_noise(samples, i, eps, z)
        if i % save_rate ==0 or i==timesteps or i<8:
            intermediate.append(samples.detach().cpu().numpy())

    intermediate = np.stack(intermediate)
    return samples, intermediate

In [18]:
# visualize samples
plt.clf()
samples, intermediate_ddpm = sample_ddpm(32)
animation_ddpm = plot_sample(intermediate_ddpm,32,4,save_dir, "ani_run", None, save=False)
HTML(animation_ddpm.to_jshtml())

d:\anaconda\envs\d2l\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


<Figure size 640x480 with 0 Axes>

#### Acknowledgements & References
[1] Ho J, Jain A, Abbeel P. Denoising diffusion probabilistic models[J]. Advances in neural information processing systems, 2020, 33: 6840-6851.  
[2] Luo C. Understanding diffusion models: A unified perspective[J]. arXiv preprint arXiv:2208.11970, 2022.  
[3] The images in this note refer to the slide of [Prof. Hung-yi Lee](https://speech.ee.ntu.edu.tw/~hylee/index.php), the tutorial video can be found at [here](https://www.bilibili.com/video/BV1mLbQeRExa/?spm_id_from=333.999.0.0).    
[4] This code is modified from https://learn.deeplearning.ai/courses/diffusion-models/lesson/jnak3/controlling.